In [1]:
# Считайте таблицу с признаками из файла features.csv с помощью кода, приведенного выше. 
# Удалите признаки, связанные с итогами матча (они помечены в описании 
# данных как отсутствующие в тестовой выборке).

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
import time
import datetime

In [2]:
data_train = pd.read_csv('features.csv')
data_test = pd.read_csv('features_test.csv')

In [3]:
test_columns = []
for i in data_test.columns:
    test_columns.append(i)
print(test_columns, len(test_columns))

['match_id', 'start_time', 'lobby_type', 'r1_hero', 'r1_level', 'r1_xp', 'r1_gold', 'r1_lh', 'r1_kills', 'r1_deaths', 'r1_items', 'r2_hero', 'r2_level', 'r2_xp', 'r2_gold', 'r2_lh', 'r2_kills', 'r2_deaths', 'r2_items', 'r3_hero', 'r3_level', 'r3_xp', 'r3_gold', 'r3_lh', 'r3_kills', 'r3_deaths', 'r3_items', 'r4_hero', 'r4_level', 'r4_xp', 'r4_gold', 'r4_lh', 'r4_kills', 'r4_deaths', 'r4_items', 'r5_hero', 'r5_level', 'r5_xp', 'r5_gold', 'r5_lh', 'r5_kills', 'r5_deaths', 'r5_items', 'd1_hero', 'd1_level', 'd1_xp', 'd1_gold', 'd1_lh', 'd1_kills', 'd1_deaths', 'd1_items', 'd2_hero', 'd2_level', 'd2_xp', 'd2_gold', 'd2_lh', 'd2_kills', 'd2_deaths', 'd2_items', 'd3_hero', 'd3_level', 'd3_xp', 'd3_gold', 'd3_lh', 'd3_kills', 'd3_deaths', 'd3_items', 'd4_hero', 'd4_level', 'd4_xp', 'd4_gold', 'd4_lh', 'd4_kills', 'd4_deaths', 'd4_items', 'd5_hero', 'd5_level', 'd5_xp', 'd5_gold', 'd5_lh', 'd5_kills', 'd5_deaths', 'd5_items', 'first_blood_time', 'first_blood_team', 'first_blood_player1', 'first

In [4]:
train_columns = []
for i in data_train.columns:
    train_columns.append(i)
print(train_columns, len(train_columns))

['match_id', 'start_time', 'lobby_type', 'r1_hero', 'r1_level', 'r1_xp', 'r1_gold', 'r1_lh', 'r1_kills', 'r1_deaths', 'r1_items', 'r2_hero', 'r2_level', 'r2_xp', 'r2_gold', 'r2_lh', 'r2_kills', 'r2_deaths', 'r2_items', 'r3_hero', 'r3_level', 'r3_xp', 'r3_gold', 'r3_lh', 'r3_kills', 'r3_deaths', 'r3_items', 'r4_hero', 'r4_level', 'r4_xp', 'r4_gold', 'r4_lh', 'r4_kills', 'r4_deaths', 'r4_items', 'r5_hero', 'r5_level', 'r5_xp', 'r5_gold', 'r5_lh', 'r5_kills', 'r5_deaths', 'r5_items', 'd1_hero', 'd1_level', 'd1_xp', 'd1_gold', 'd1_lh', 'd1_kills', 'd1_deaths', 'd1_items', 'd2_hero', 'd2_level', 'd2_xp', 'd2_gold', 'd2_lh', 'd2_kills', 'd2_deaths', 'd2_items', 'd3_hero', 'd3_level', 'd3_xp', 'd3_gold', 'd3_lh', 'd3_kills', 'd3_deaths', 'd3_items', 'd4_hero', 'd4_level', 'd4_xp', 'd4_gold', 'd4_lh', 'd4_kills', 'd4_deaths', 'd4_items', 'd5_hero', 'd5_level', 'd5_xp', 'd5_gold', 'd5_lh', 'd5_kills', 'd5_deaths', 'd5_items', 'first_blood_time', 'first_blood_team', 'first_blood_player1', 'first

In [5]:
train = set(train_columns)
test = set(test_columns)
train.difference(test)

{'barracks_status_dire',
 'barracks_status_radiant',
 'duration',
 'radiant_win',
 'tower_status_dire',
 'tower_status_radiant'}

In [6]:
data_test.shape

(17177, 103)

In [7]:
data_train.shape

(97230, 109)

In [9]:
# Проверьте выборку на наличие пропусков с помощью функции count(), которая для каждого столбца 
# показывает число заполненных значений. Много ли пропусков в данных? Запишите названия признаков, 
# имеющих пропуски, и попробуйте для любых двух из них дать обоснование, почему их значения могут быть пропущены.

In [8]:
data_train.count()

match_id                       97230
start_time                     97230
lobby_type                     97230
r1_hero                        97230
r1_level                       97230
r1_xp                          97230
r1_gold                        97230
r1_lh                          97230
r1_kills                       97230
r1_deaths                      97230
r1_items                       97230
r2_hero                        97230
r2_level                       97230
r2_xp                          97230
r2_gold                        97230
r2_lh                          97230
r2_kills                       97230
r2_deaths                      97230
r2_items                       97230
r3_hero                        97230
r3_level                       97230
r3_xp                          97230
r3_gold                        97230
r3_lh                          97230
r3_kills                       97230
r3_deaths                      97230
r3_items                       97230
r

In [9]:
columns_empty = []
for name, amount in enumerate(data_train.count()):
    if amount<97230:
        columns_empty.append(name)
print(columns_empty)

[83, 84, 85, 86, 87, 88, 89, 94, 95, 96, 97, 102]


In [10]:
col_empty_names = []
for i, value in enumerate(data_train):
    for k in columns_empty:
        if i == k:
            col_empty_names.append(value)
print(col_empty_names)

['first_blood_time', 'first_blood_team', 'first_blood_player1', 'first_blood_player2', 'radiant_bottle_time', 'radiant_courier_time', 'radiant_flying_courier_time', 'radiant_first_ward_time', 'dire_bottle_time', 'dire_courier_time', 'dire_flying_courier_time', 'dire_first_ward_time']


In [13]:
# Замените пропуски на нули с помощью функции fillna(). На самом деле этот способ является 
# предпочтительным для логистической регрессии, поскольку он позволит пропущенному значению не 
# вносить никакого вклада в предсказание. Для деревьев часто лучшим вариантом оказывается замена пропуска 
# на очень большое или очень маленькое значение — в этом случае при построении разбиения вершины можно будет 
# отправить объекты с пропусками в отдельную ветвь дерева. Также есть и другие подходы — например, замена 
# пропуска на среднее значение признака. Мы не требуем этого в задании, но при желании попробуйте разные подходы 
# к обработке пропусков и сравните их между собой.

In [11]:
data_train.fillna(0)

,match_id,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,...,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time,duration,radiant_win,tower_status_radiant,tower_status_dire,barracks_status_radiant,barracks_status_dire
0,0,1430198770,7,11,5,2098,1489,20,0,0,...,4,2,2,-52.0,2874,1,1796,0,51,0
1,1,1430220345,0,42,4,1188,1033,9,0,1,...,4,3,1,-5.0,2463,1,1974,0,63,1
2,2,1430227081,7,33,4,1319,1270,22,0,0,...,4,3,1,13.0,2130,0,0,1830,0,63
3,3,1430263531,1,29,4,1779,1056,14,0,0,...,4,2,0,27.0,1459,0,1920,2047,50,63
4,4,1430282290,7,13,4,1431,1090,8,1,0,...,3,3,0,-16.0,2449,0,4,1974,3,63
5,5,1430284186,1,11,5,1961,1461,19,0,1,...,4,4,0,-43.0,1453,0,512,2038,0,63
6,8,1430293701,1,8,3,967,1136,7,1,0,...,6,3,0,10.0,1968,0,1536,1983,12,63
7,9,1430299335,7,35,5,2117,1252,16,0,0,...,3,4,0,-15.0,4079,1,1540,0,63,0
8,11,1430308974,1,17,5,1527,906,10,0,1,...,3,4,0,26.0,3071,0,0,1572,0,63
9,12,1430316105,7,15,5,1651,1060,14,0,1,...,3,2,0,16.0,2384,0,0,2038,0,63


In [15]:
# Какой столбец содержит целевую переменную? Запишите его название 
radiant_win

NameError: name 'radiant_win' is not defined

In [ ]:
# Забудем, что в выборке есть категориальные признаки, и 
# попробуем обучить градиентный бустинг над деревьями на имеющейся матрице "объекты-признаки". 
# Зафиксируйте генератор разбиений для кросс-валидации по 5 блокам (KFold), не забудьте перемешать при этом выборку 
# (shuffle=True), поскольку данные в таблице отсортированы по времени, и без перемешивания можно столкнуться с 
# нежелательными эффектами при оценивании качества. Оцените качество градиентного бустинга 
# (GradientBoostingClassifier) с помощью данной кросс-валидации, попробуйте при этом разное количество деревьев 
# (как минимум протестируйте следующие значения для количества деревьев: 10, 20, 30). 
# Долго ли настраивались классификаторы? Достигнут ли оптимум на испытанных значениях параметра n_estimators, 
# или же качество, скорее всего, продолжит расти при дальнейшем его увеличении?

In [12]:
y_train = data_train['radiant_win']

In [13]:
X_train = data_train.drop(train.difference(test), axis=1).fillna(0)

In [14]:
X_train

,match_id,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,...,radiant_ward_sentry_count,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time
0,0,1430198770,7,11,5,2098,1489,20,0,0,...,0,35.0,103.0,-84.0,221.0,3,4,2,2,-52.0
1,1,1430220345,0,42,4,1188,1033,9,0,1,...,0,-20.0,149.0,-84.0,195.0,5,4,3,1,-5.0
2,2,1430227081,7,33,4,1319,1270,22,0,0,...,1,-39.0,45.0,-77.0,221.0,3,4,3,1,13.0
3,3,1430263531,1,29,4,1779,1056,14,0,0,...,0,-30.0,124.0,-80.0,184.0,0,4,2,0,27.0
4,4,1430282290,7,13,4,1431,1090,8,1,0,...,0,46.0,182.0,-80.0,225.0,6,3,3,0,-16.0
5,5,1430284186,1,11,5,1961,1461,19,0,1,...,1,-27.0,2.0,-86.0,212.0,4,4,4,0,-43.0
6,8,1430293701,1,8,3,967,1136,7,1,0,...,2,-35.0,6.0,-86.0,182.0,3,6,3,0,10.0
7,9,1430299335,7,35,5,2117,1252,16,0,0,...,1,26.0,119.0,-61.0,0.0,2,3,4,0,-15.0
8,11,1430308974,1,17,5,1527,906,10,0,1,...,1,9.0,0.0,-84.0,226.0,1,3,4,0,26.0
9,12,1430316105,7,15,5,1651,1060,14,0,1,...,0,-24.0,186.0,-83.0,195.0,1,3,2,0,16.0


In [15]:
kf = KFold(n_splits=5, shuffle=True, random_state=241)

In [16]:
arr = [10,20,30]

In [17]:
X_test = data_test.fillna(0)

In [18]:
X_train.shape

(97230, 103)

In [19]:
y_train.shape

(97230,)

In [20]:
X_test.shape

(17177, 103)

In [25]:
start_time = datetime.datetime.now()
time.sleep(3)
grid = {'n_estimators': [5, 10,20,30,40]}
GBC = GradientBoostingClassifier(random_state=241)
gs = GridSearchCV(GBC, grid, scoring='roc_auc', cv=kf)
gs.fit(X_train, y_train)
print ('Time elapsed:', datetime.datetime.now() - start_time)

Time elapsed: 0:06:18.710298


In [26]:
gs.best_estimator_

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=40,
              n_iter_no_change=None, presort='auto', random_state=241,
              subsample=1.0, tol=0.0001, validation_fraction=0.1,
              verbose=0, warm_start=False)

In [27]:
gs.best_params_

{'n_estimators': 40}

In [28]:
gs.best_score_

0.6941311257119158

In [29]:
arr = [5, 10, 20, 30, 40]

In [30]:
start_time = datetime.datetime.now()
time.sleep(3)
estimates = []
for i in arr:
    GBC = GradientBoostingClassifier(n_estimators=i, random_state=241)
    GBC.fit(X_train, y_train)
    pred = GBC.predict_proba(X_train)[:, 1]
    roc = roc_auc_score(y_train, pred)
    estimates.append(roc)
print(estimates)
print ('Time elapsed:', datetime.datetime.now() - start_time)

[0.6416681014148848, 0.6724523413243607, 0.6915885371211129, 0.7002088803900559, 0.7055866294138532]
Time elapsed: 0:01:20.843922


In [31]:
arr = [30,40,50,60]

# Логистическая регрессия

In [32]:
# Оцените качество логистической регрессии (sklearn.linear_model.LogisticRegression с L2-регуляризацией) 
# с помощью кросс-валидации по той же схеме, которая использовалась для градиентного бустинга. Подберите при 
# этом лучший параметр регуляризации (C). Какое наилучшее качество у вас получилось? Как оно соотносится с 
# качеством градиентного бустинга? Чем вы можете объяснить эту разницу? Быстрее ли работает логистическая 
# регрессия по сравнению с градиентным бустингом?

In [22]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

C:\Users\ivan1\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\ivan1\Anaconda3\lib\site-packages\sklearn\base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [23]:
X_train_scaled

array([[-1.73252632, -2.54436416,  1.54068827, ..., -0.55115386,
         1.84600409, -1.12149424],
       [-1.73249602, -2.54045236, -0.92779756, ...,  0.67817009,
         0.43778816,  0.04394713],
       [-1.73246573, -2.53923104,  1.54068827, ...,  0.67817009,
         0.43778816,  0.49028637],
       ...,
       [ 1.73353115,  1.09874571, -0.57515673, ...,  0.67817009,
         0.43778816, -0.20401912],
       [ 1.73356145,  1.09895204, -0.57515673, ...,  0.67817009,
         0.43778816, -0.87352799],
       [ 1.73359175,  1.1026479 ,  1.54068827, ..., -0.55115386,
        -0.97042777, -0.79913812]])

In [29]:
grid = {'C': np.power(10.0, np.arange(-70, 10))}
kf = KFold(n_splits=5, shuffle=True, random_state=241)
start_time = datetime.datetime.now()
time.sleep(3)
clf = LogisticRegression(penalty = 'l2',random_state=241, solver = 'lbfgs')
gs = GridSearchCV(clf, grid, scoring = 'roc_auc', cv=kf)
gs.fit(X_train_scaled, y_train.values)
print ('Time elapsed:', datetime.datetime.now() - start_time)

Time elapsed: 0:02:45.426807


In [30]:
gs.best_estimator_

LogisticRegression(C=0.01, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=241, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)

In [31]:
gs.best_score_

0.7163798939991932

In [32]:
gs.best_params_

{'C': 0.01}

In [37]:
# Среди признаков в выборке есть категориальные, которые мы использовали как числовые, 
# что вряд ли является хорошей идеей. Категориальных признаков в этой задаче одиннадцать: 
# lobby_type и r1_hero, r2_hero, ..., r5_hero, d1_hero, d2_hero, ..., d5_hero. 
# Уберите их из выборки, и проведите кросс-валидацию для логистической регрессии на новой выборке с подбором 
# лучшего параметра регуляризации. Изменилось ли качество? Чем вы можете это объяснить?

In [39]:
categorial = ['lobby_type', 'r1_hero', 'r2_hero', 'r3_hero', 'r4_hero', 'r5_hero', 'd1_hero', 'd2_hero',
             'd3_hero', 'd4_hero', 'd5_hero']

In [40]:
X_train = X_train.drop(categorial, axis = 1)

KeyError: "['lobby_type' 'r1_hero' 'r2_hero' 'r3_hero' 'r4_hero' 'r5_hero' 'd1_hero'\n 'd2_hero' 'd3_hero' 'd4_hero' 'd5_hero'] not found in axis"

In [41]:
X_train.head()

,match_id,start_time,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,r2_level,...,radiant_ward_sentry_count,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time
0,0,1430198770,5,2098,1489,20,0,0,7,3,...,0,35.0,103.0,-84.0,221.0,3,4,2,2,-52.0
1,1,1430220345,4,1188,1033,9,0,1,12,4,...,0,-20.0,149.0,-84.0,195.0,5,4,3,1,-5.0
2,2,1430227081,4,1319,1270,22,0,0,12,3,...,1,-39.0,45.0,-77.0,221.0,3,4,3,1,13.0
3,3,1430263531,4,1779,1056,14,0,0,5,2,...,0,-30.0,124.0,-80.0,184.0,0,4,2,0,27.0
4,4,1430282290,4,1431,1090,8,1,0,8,2,...,0,46.0,182.0,-80.0,225.0,6,3,3,0,-16.0


In [42]:
X_train_scaled = scaler.fit_transform(X_train)

C:\Users\ivan1\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\ivan1\Anaconda3\lib\site-packages\sklearn\base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [46]:
start_time = datetime.datetime.now()
time.sleep(3)
grid = {'C': np.power(10.0, np.arange(-70, 10))}
kf = KFold(n_splits=5, shuffle=True, random_state=241)
clf = LogisticRegression(penalty = 'l2',random_state=241, solver = 'lbfgs')
gs = GridSearchCV(clf, grid, scoring = 'roc_auc', cv=kf)
gs.fit(X_train_scaled, y_train)
print ('Time elapsed:', datetime.datetime.now() - start_time)

Time elapsed: 0:02:31.324747


In [26]:
gs.best_estimator_

AttributeError: 'GridSearchCV' object has no attribute 'best_estimator_'

In [25]:
gs.best_score_

AttributeError: 'GridSearchCV' object has no attribute 'best_score_'

In [47]:
gs.best_params_

{'C': 1e-32}

In [48]:
# На предыдущем шаге мы исключили из выборки признаки rM_hero и dM_hero, которые показывают, 
# какие именно герои играли за каждую команду. Это важные признаки — герои имеют разные характеристики, 
# и некоторые из них выигрывают чаще, чем другие. Выясните из данных, сколько различных идентификаторов 
# героев существует в данной игре (вам может пригодиться фукнция unique или value_counts).

In [49]:
data_train['r2_hero'].value_counts()

7      4201
11     4128
112    3639
50     3477
72     3468
39     3008
100    2812
71     2581
55     2456
21     2433
30     2205
25     2176
85     2073
28     1980
26     1910
106    1900
51     1844
86     1706
68     1699
8      1692
87     1630
60     1533
47     1212
75     1197
74     1175
46     1130
42     1085
1      1080
3      1075
99     1052
       ... 
14      302
43      298
89      292
33      284
90      270
9       268
79      258
92      245
63      243
34      229
54      210
23      209
35      200
45      198
95      190
65      189
76      177
83      172
32      168
105     164
82      158
53      154
78      134
10      103
77       97
66       77
80       69
58       67
103      62
109      18
Name: r2_hero, Length: 108, dtype: int64

In [50]:
len(data_train['d1_hero'].unique())

108

In [51]:
# Воспользуемся подходом "мешок слов" для кодирования информации о героях. Пусть всего в игре имеет N различных 
# героев. Сформируем N признаков, при этом i-й будет равен нулю, если i-й герой не участвовал в матче; единице, 
# если i-й герой играл за команду Radiant; минус единице, если i-й герой играл за команду Dire. 
# Ниже вы можете найти код, который выполняет данной преобразование. Добавьте полученные признаки к числовым, 
# которые вы использовали во втором пункте данного этапа.

In [63]:
categ = ['r1_hero', 'r2_hero', 'r3_hero', 'r4_hero', 'r5_hero', 'd1_hero', 'd2_hero',
             'd3_hero', 'd4_hero', 'd5_hero']

In [64]:
data = data_train[categ]

In [80]:
X_pick = np.zeros((data.shape[0], 108))

for i, match_id in enumerate(data.index):
    for p in range(5):
        X_pick[i, data.loc[match_id, 'r%d_hero' % (p+1)]-5] = 1
        X_pick[i, data.loc[match_id, 'd%d_hero' % (p+1)]-5] = -1

In [82]:
X_pick.shape

(97230, 108)

In [83]:
X_train.shape

(97230, 103)

In [108]:
import numpy as np

In [109]:
X = np.hstack((X_train.values, X_pick))

In [110]:
grid = {'C': np.power(10.0, np.arange(-70, 10))}
kf = KFold(n_splits=5, shuffle=True, random_state=241)
clf = LogisticRegression(penalty = 'l2',random_state=241)
gs = GridSearchCV(clf, grid, scoring = 'roc_auc', cv=kf)
gs.fit(new_1, y_train_new)
print ('Time elapsed:', datetime.datetime.now() - start_time)

C:\Users\ivan1\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\ivan1\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\ivan1\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\ivan1\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\ivan1\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence t

C:\Users\ivan1\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\ivan1\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\ivan1\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\ivan1\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\ivan1\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence t

C:\Users\ivan1\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\ivan1\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\ivan1\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\ivan1\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\ivan1\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence t

C:\Users\ivan1\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\ivan1\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\ivan1\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\ivan1\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\ivan1\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence t

C:\Users\ivan1\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\ivan1\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\ivan1\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\ivan1\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\ivan1\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence t

...

C:\Users\ivan1\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\ivan1\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\ivan1\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\ivan1\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\ivan1\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence t

C:\Users\ivan1\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\ivan1\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\ivan1\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\ivan1\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\ivan1\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence t

C:\Users\ivan1\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\ivan1\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\ivan1\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\ivan1\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\ivan1\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence t

C:\Users\ivan1\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Time elapsed: 1:45:47.539059


In [111]:
gs.best_estimator_

LogisticRegression(C=1e-32, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=241, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [112]:
gs.best_score_

0.5134556599957437

In [113]:
gs.best_params_

{'C': 1e-32}